In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import minimize
import glob
import os
import sys

from read_ybc import readYBC
from read_stars import read_manual_stars
from wcs_utils import apply_wcs_to_photometry, radec2altaz

from lsst.all_sky_phot.wcs import wcs_zea, wcs_refine_zea, fit_xyshifts, Fisheye, distortion_mapper
from lsst.all_sky_phot import phot_night, readcr2

from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS, Longitude, Latitude
import astropy.units as u
from astropy.time import Time
from astropy.table import Table, vstack
from astropy import units as u

from photutils import CircularAperture

In [2]:
# Now to run photometry on a full image and refine the WCS solution
# Load the Yale bright star catalog
ybc = readYBC()

In [3]:
# Load up the photometry tables from the 1st night
temp = np.load('012716_night_phot.npz')
phot_tables = temp['phot_tables'][()]
temp.close()

In [4]:
# read in the rough WCS
# the simple wcs based on just identified stars
hdulist = fits.open('wcs_refined.fits')
wcs_refined = wcs.WCS(hdulist[0].header)
hdulist.close()

In [5]:
# Generate cataog, alt, az, mjd.
alts = []
azs = []
mjds = []
observed_x = []
observed_y = []
observed_mjd = []
lsst_location = EarthLocation(lat=-30.2444*u.degree, lon=-70.7494*u.degree, height=2650.0*u.meter)
for phot_table in phot_tables[50:55]:
    mjd = phot_table['mjd'][0]
    alt, az = radec2altaz(ybc['RA'], ybc['Dec'], mjd, location=lsst_location)
    good = np.where(alt > 0.)
    alts.append(alt[good])
    azs.append(az[good])
    mjds.append(az[good]*0+mjd)
    
    observed_x.append(phot_table['xcenter'].value)
    observed_y.append(phot_table['ycenter'].value)
    observed_mjd.append(phot_table['mjd'].data)

/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/astropy/coordinates/angles.py:672: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/astropy/coordinates/angles.py:672: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/astropy/coordinates/angles.py:563: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/astropy/coordinates/angles.py:563: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):
If you need enough precisi

In [6]:
alts = np.concatenate(alts)
azs = np.concatenate(azs)
mjds = np.concatenate(mjds)

observed_x = np.concatenate(observed_x)
observed_y = np.concatenate(observed_y)
observed_mjd = np.concatenate(observed_mjd)

In [7]:
u_coord, v_coord = wcs_refined.all_world2pix(azs, alts, 0)

In [8]:
# (3870, 5796)
xgrid = np.linspace(0,5796, 35)
ygrid = np.linspace(0,3870, 25)
xgrid, ygrid = np.meshgrid(xgrid, ygrid)

In [9]:
xp = []
yp = []
fits = []
i=0
imax = float(xgrid.size*ygrid.size)
for u_center in xgrid.ravel():
    for v_center in ygrid.ravel():
        xp.append(u_center)
        yp.append(v_center)
        fit_result = distortion_mapper(observed_x, observed_y, observed_mjd, alts, azs,
                                       mjds, wcs_refined,window=100, u_center=u_center, v_center=v_center)
        fits.append(fit_result)
        i += 1
        progress = i/imax*100
        text = "\rprogress = %.2f%%" % progress
        sys.stdout.write(text)
        sys.stdout.flush()

progress = 1.10%

KeyboardInterrupt: 

In [ ]:
xoff = []
yoff = []
for fit in fits:
    if fit is not None:
        xoff.append(fit.x[0])
        yoff.append(fit.x[1])
    else:
        xoff.append(0)
        yoff.append(0)

In [ ]:
plt.scatter(xp, yp, c=xoff)
cb = plt.colorbar()

In [ ]:
plt.scatter(xp, yp, c=yoff)
cb = plt.colorbar()

In [11]:
len(phot_tables)

91